Download the following data source: Souvenirs Data from the April 2021 Dev10 Data Professional program. You will need to unzip the file to work with the CSVs directly.
Using souvenir.csv as the base:
1. Read in all of the other CSVs

In [44]:
import pandas as pd
souvenirs=pd.read_csv('souvenir.csv')
categories=pd.read_csv('category.csv')
cities=pd.read_csv('city.csv')
countries=pd.read_csv('country.csv')
locations=pd.read_csv('location.csv')
owners=pd.read_csv('owner.csv')
regions=pd.read_csv('region.csv')
souvenirs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ItemID           147 non-null    int64  
 1   LocationID       31 non-null     float64
 2   ItemDescription  109 non-null    object 
 3   DateObtained     147 non-null    object 
 4   Price            147 non-null    float64
 5   Weight           109 non-null    float64
 6   OwnerID          147 non-null    int64  
 7   CategoryID       147 non-null    int64  
 8   CityID           125 non-null    float64
 9   Name             147 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 11.6+ KB


2. Replace the ID fields with the descriptions

In [43]:
souvenirs.rename(columns={"Name":"SouvenirName"},inplace=True)
categories.rename(columns={"Name":"CategoryName"},inplace=True)
owners.rename(columns={"Name":"OwnerName"},inplace=True)
regions_with_countries=regions.merge(countries,how="inner",on="CountryID")
regions_with_countries.rename(columns={"name":"CountryName","Name":"RegionName"},inplace=True)
regions_with_countries.drop('CountryID',inplace=True,axis=1)
cities_with_RC=cities.merge(regions_with_countries,how="inner",on="RegionID")
cities_with_RC.rename(columns={"Name":"CityName"},inplace=True)
cities_with_RC.drop('RegionID',inplace=True,axis=1)
souvenirs_with_CRC=souvenirs.merge(cities_with_RC,how="left",on="CityID")
souvenirs_with_catCRC=souvenirs_with_CRC.merge(categories,how='inner',on='CategoryID')
souvenirs_with_catCRCL=souvenirs_with_catCRC.merge(locations,how="left",on='LocationID')
souvenirs_with_all=souvenirs_with_catCRCL.merge(owners,how='inner',on='OwnerID')
souvenirs_with_all.drop(['LocationID','OwnerID','CategoryID','CityID'],inplace=True,axis=1)
souvenirs_with_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 0 to 146
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ItemID           147 non-null    int64  
 1   ItemDescription  109 non-null    object 
 2   DateObtained     147 non-null    object 
 3   Price            147 non-null    float64
 4   Weight           109 non-null    float64
 5   SouvenirName     147 non-null    object 
 6   CityName         125 non-null    object 
 7   RegionName       125 non-null    object 
 8   CountryName      125 non-null    object 
 9   CategoryName     147 non-null    object 
 10  PlaceName        31 non-null     object 
 11  Latitude         20 non-null     float64
 12  Longitude        20 non-null     float64
 13  OwnerName        147 non-null    object 
dtypes: float64(4), int64(1), object(9)
memory usage: 17.2+ KB


3. Save the new file to souvenirs.JSON

In [47]:
import json
souvenirsjson = souvenirs_with_all.to_json()
file=open('souvenirs json test.json','w') # open the file to edit

json.dump(souvenirsjson,file)
file.close()

4. Create separate CSV files for each owner's set of souvenirs.

In [48]:
owners_list=souvenirs_with_all.OwnerName.unique()
for owner in owners_list:
    souvenirs_with_all[souvenirs_with_all.OwnerName==owner].to_csv(f'{owner} souvenirs.csv')

Pandas supports ZIP files. Look into how Pandas works with ZIP files and see how that might change how you handle this exercise.